## resnet18 and resnet34

In [26]:
import torch
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, input_channels, output_channels, downsample=None, stride=(1,1)) -> None:
        super(ConvBlock, self).__init__()

        self.conv1 = nn.Conv2d(input_channels, output_channels, kernel_size=(3,3), stride=stride, padding=(1,1))
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels, output_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = downsample

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, input_channels, layers, num_classes=10) -> None:
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3))
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2), padding=(1,1))

        self.layer1 = self.ensamble_blocks(block, 64, layers[0], stride=(1,1))
        self.layer2 = self.ensamble_blocks(block, 128, layers[1], stride=(2,2))
        self.layer3 = self.ensamble_blocks(block, 256, layers[2], stride=(2,2))
        self.layer4 = self.ensamble_blocks(block, 512, layers[3], stride=(2,2))

        self.avg = nn.AvgPool2d(kernel_size=(7,7), stride=(1,1))
        self.fc = nn.Linear(512, num_classes)


    def ensamble_blocks(self, block, output_channels, residual_blocks, stride=(1,1)):
        identity_downsample = None
        layers = []

        if stride != (1,1) or self.in_channels != output_channels:
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, output_channels, kernel_size=(1,1), stride=stride),
                nn.BatchNorm2d(output_channels)
            )
        
        layers.append(block(self.in_channels, output_channels, identity_downsample, stride))
        
        self.in_channels = output_channels

        for i in range(residual_blocks):
            layers.append(block(self.in_channels, output_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg(x)
        x = self.fc(x.reshape(x.shape[0], -1))
        return x

In [28]:
structure = {
    "ResNet18": [2, 2, 2, 2],
    "ResNet34": [3, 4, 6, 3],
}

ResNet18 = ResNet(ConvBlock, input_channels=3, layers=structure["ResNet18"], num_classes=10)

x = torch.rand(1,3,224,224)

print(ResNet18(x).shape)

torch.Size([1, 10])


## resnet50 resnet101 and resnet152

In [29]:
import torch
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, input_channels, output_channels, downsample=None, stride=(1,1)) -> None:
        super(ConvBlock, self).__init__()

        self.expansion = 4
        self.conv1 = nn.Conv2d(input_channels, output_channels, kernel_size=(1,1), stride=(1,1), padding=(0,0))
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels, output_channels, kernel_size=(3,3), stride=stride, padding=(1,1))
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.conv3 = nn.Conv2d(output_channels, output_channels*self.expansion, kernel_size=(1,1), stride=(1,1), padding=(0,0))
        self.bn3 = nn.BatchNorm2d(output_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = downsample

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, input_channels, layers, num_classes=10) -> None:
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3))
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2), padding=(1,1))

        self.layer1 = self.ensamble_blocks(block, 64, layers[0], stride=(1,1))
        self.layer2 = self.ensamble_blocks(block, 128, layers[1], stride=(2,2))
        self.layer3 = self.ensamble_blocks(block, 256, layers[2], stride=(2,2))
        self.layer4 = self.ensamble_blocks(block, 512, layers[3], stride=(2,2))

        self.avg = nn.AvgPool2d(kernel_size=(7,7), stride=(1,1))
        self.fc = nn.Linear(512*4, num_classes)


    def ensamble_blocks(self, block, output_channels, residual_blocks, stride=(1,1)):
        identity_downsample = None
        layers = []

        if stride != (1,1) or self.in_channels != output_channels*4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, output_channels*4, kernel_size=(1,1), stride=stride),
                nn.BatchNorm2d(output_channels*4)
            )
        
        layers.append(block(self.in_channels, output_channels, identity_downsample, stride))
        
        self.in_channels = output_channels*4

        for i in range(residual_blocks):
            layers.append(block(self.in_channels, output_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg(x)
        x = self.fc(x.reshape(x.shape[0], -1))
        return x

In [32]:
structure = {
    "ResNet50": [3, 4, 6, 3],
    "ResNet101": [3, 4, 23, 3],
    "ResNet152": [3, 8, 36, 3]
}

ResNet50 = ResNet(ConvBlock, input_channels=3, layers=structure["ResNet50"], num_classes=10)

x = torch.rand(1,3,224,224)

print(ResNet50(x).shape)

torch.Size([1, 10])
